In [122]:
import torch
import time
import pandas as pd
import numpy as np
import re
import gensim
import collections
import nltk
from nltk import word_tokenize
from stopwordsallforms import STOPWORDS as arb_stopwords
import elements
#nltk.download("stopwords")
nltk.download('punkt')
import pandas as pd

[nltk_data] Downloading package punkt to /home/amr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data
### Helping Functions

In [123]:
def normalization(t):
    return t.translate(t.maketrans(''.join(elements.ALEF_HAMZA_FORMS+elements.NON_ALIF_HAMZA_FORMS+tuple(elements.ALIF_MAQSURA)),"اااويي"))

In [124]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

### Data

In [127]:
data= np.load('../translation project/AD_NMT-master/LAV-MSA-2-both.pkl',allow_pickle=True)

In [128]:
data[0] # lav , msa

['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش',
 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة']

In [129]:
# extract only msa text
msa=[]
for i,ex in enumerate(data):
    msa_text = stopwords_removal(clean_str(normalization(ex[1])))
    data[i][1] = msa_text
    msa.append(msa_text)

In [130]:
msa = ' '.join(msa)

Dictionaries

In [131]:
msa_d=collections.Counter(msa.split())

In [132]:
min_count = 2

In [133]:
idx2msa = np.array([word for word,freq in msa_d.items() if freq > min_count ])

In [134]:
msa2idx = {word:i for i,word in enumerate(idx2msa)}

In [136]:
msa_data = [' '.join([i for i in t[1].split() if (msa2idx.get(i,-1) != -1 and t[1] != '')]) for t in data]

In [139]:
msa_data = [i for i in msa_data if i != '']

Load Embeddings

In [35]:
t_model = gensim.models.Word2Vec.load('../resources/models/word vectors/word2vec/wiki/full_grams_cbow_100_wiki/full_grams_cbow_100_wiki.mdl')

In [37]:
i2l = list(elements.BASIC_LETTERS)
i2v = {}
for letter in i2l:
    i2v[i] = t_model.wv.get_vector(letter)
i2l +=  ' '
l2i = {v:i for i,v in enumerate(i2l)}

Deep Learning

In [141]:
from torch.utils.data import Dataset,DataLoader

In [142]:
from torch.nn.utils.rnn import pad_sequence 

In [143]:
def noise(txt):
    print('txt',txt)
    noise_sz = np.random.randint(0,len(txt),1)
    replace_idx = np.random.choice(len(txt),noise_sz,replace=False)
    letters_idx = np.random.choice(len(i2l),noise_sz,replace=True)
    txt = list(txt)
    for rep,let in zip(replace_idx,letters_idx):
        txt[rep] = i2l[let]
    return ''.join(txt)

In [144]:
c=0
for i in msa_data:
    if i=='' :
        c+=1

In [145]:
class arrDs(Dataset):
    def __init__(self,txt_list,l2i):
        self.data = txt_list
        self.l2i = l2i
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        X = noise(self.data[idx])
        Y = self.data[idx]
        
        X = torch.tensor([self.l2i[i] for i in X])
        Y = torch.tensor([self.l2i[i] for i in Y])
        #numerilize
        return (X,Y)

In [146]:
ds = arrDs(msa_data,l2i)

In [147]:
def collate_fn(data):
    data,label  = data
    #data = [torch.tensor(i).clone().detach() for i in data]
    data = pad_sequence(data,batch_first=True)
    return data

In [148]:
dl = DataLoader(ds,batch_size=2,collate_fn=collate_fn)

In [149]:
for i in dl:
    pass

txt اعرف واحده ذهبت فرنسا غرفه وضعت
txt اذهب تقدم يسارا
txt يجب يكون موضوع
txt تبتعد تفقد الطريق
txt اقصد صراحه ياامي ايضا كرهته يوم حضرته مثلما خالتي كرهته
txt قال موضوعك جيد ساقول كنت
txt كنت اردت اخوتي اشتري الطيبه اخر الليل متاكده تصبح
txt ايضا عندما بدوا المنزل مرضت لماذا
txt كلمات جميله كليب جميل ويحتوي تشجيع لبس الحجاب الحقيقه تعجبت الكلام الغريب تحدثتي الاغنيه
txt العرض الامر وظيفه محترمه تعني محل للسكن وسياره
txt يخاف الحرب يخاف شي مهم يخسره
txt الجزاير العاصمه
txt الخاصه قليله
txt المشكله
txt اعملي جيدا
txt جيت تعرف يجب تفعله تحسبين العمل سهل بلادنا
txt السلامه
txt اكتب حاليا
txt اسمعها
txt قالت مراد قال سامي
txt يقول تفعله قال وجدتك تحاول تفكها ففعلت بدلا
txt المهم يكون
txt قالت عايده المنزل
txt اخذت الدوله
txt اظن سليمه
txt فعلنا
txt بدات الحلقه
txt تم ترسيمها
txt المسكينه عانت كثيرا
txt اداره
txt مازالت البلد يتعب ويشقي الشباب
txt اخترت فيفري المقايس
txt المشكله تكمن النظام باكمله
txt شيء


KeyError: 'ء'